# Load Tic-Tac-Toe Data to ClickHouse

This notebook reads all CSV files from the data_5x5 folder and loads them into ClickHouse database.


In [ ]:
# Import required libraries
import os
import pandas as pd
from clickhouse_driver import Client
import glob
from tqdm import tqdm
import re


In [ ]:
# ClickHouse connection configuration
CLICKHOUSE_HOST = 'localhost'
CLICKHOUSE_PORT = 9000
CLICKHOUSE_USER = 'admin'
CLICKHOUSE_PASSWORD = 'Helloworld'
CLICKHOUSE_DATABASE = 'tictactoe'

# Data directory
DATA_DIR = '/Users/nguyendinhkhiem/Development/Python/TicTacToeAI/data/data/data_5x5'

# Batch size for inserting data (adjust based on memory)
BATCH_SIZE = 100000


In [ ]:
# Connect to ClickHouse
try:
    client = Client(
        host=CLICKHOUSE_HOST,
        port=CLICKHOUSE_PORT,
        user=CLICKHOUSE_USER,
        password=CLICKHOUSE_PASSWORD
    )
    print("Connected to ClickHouse successfully!")
    print(f"Server version: {client.connection.server_info.version_major}.{client.connection.server_info.version_minor}")
except Exception as e:
    print(f"Error connecting to ClickHouse: {e}")
    raise


In [ ]:
# Create database if it doesn't exist
client.execute(f"CREATE DATABASE IF NOT EXISTS {CLICKHOUSE_DATABASE}")
print(f"Database '{CLICKHOUSE_DATABASE}' created or already exists.")


In [ ]:
# Create table for tic-tac-toe data
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {CLICKHOUSE_DATABASE}.tic_tac_toe_states (
    layer_name String,
    i11 UInt8,
    i12 UInt8,
    i13 UInt8,
    i14 UInt8,
    i15 UInt8,
    i21 UInt8,
    i22 UInt8,
    i23 UInt8,
    i24 UInt8,
    i25 UInt8,
    i31 UInt8,
    i32 UInt8,
    i33 UInt8,
    i34 UInt8,
    i35 UInt8,
    i41 UInt8,
    i42 UInt8,
    i43 UInt8,
    i44 UInt8,
    i45 UInt8,
    i51 UInt8,
    i52 UInt8,
    i53 UInt8,
    i54 UInt8,
    i55 UInt8
) ENGINE = MergeTree()
ORDER BY (layer_name)
"""

client.execute(create_table_query)
print("Table 'tic_tac_toe_states' created or already exists.")


In [ ]:
# Get all CSV files sorted by layer number
csv_files = sorted(
    glob.glob(os.path.join(DATA_DIR, 'tic_tac_toe_layer_*.csv')),
    key=lambda x: int(re.search(r'layer_(\d+)', x).group(1))
)

print(f"Found {len(csv_files)} CSV files to process:")
for csv_file in csv_files:
    file_size = os.path.getsize(csv_file) / (1024 ** 2)  # Convert to MB
    print(f"  - {os.path.basename(csv_file)} ({file_size:.2f} MB)")


In [ ]:
# Function to insert data in batches
def insert_batch(client, database, table, data_batch, layer_name):
    """
    Insert a batch of data into ClickHouse
    """
    if not data_batch:
        return
    
    # Prepare data with layer_name
    data_with_layer = [[layer_name] + row for row in data_batch]
    
    # Insert data
    insert_query = f"INSERT INTO {database}.{table} VALUES"
    client.execute(insert_query, data_with_layer)

print("Helper function defined.")


In [ ]:
# Process each CSV file
total_rows_inserted = 0

for csv_file in csv_files:
    # Extract layer name from filename
    layer_name = re.search(r'tic_tac_toe_(layer_\d+)', os.path.basename(csv_file)).group(1)
    
    print(f"\nProcessing {layer_name}...")
    
    # Count total lines for progress bar
    with open(csv_file, 'r') as f:
        total_lines = sum(1 for _ in f)
    
    # Read and insert data in batches
    batch = []
    rows_processed = 0
    
    with open(csv_file, 'r') as f:
        with tqdm(total=total_lines, desc=f"Loading {layer_name}") as pbar:
            for line in f:
                # Parse the line (comma-separated values)
                values = [int(x.strip()) for x in line.strip().split(',')]
                
                if len(values) == 25:  # Ensure we have exactly 25 values
                    batch.append(values)
                    rows_processed += 1
                    
                    # Insert batch when it reaches BATCH_SIZE
                    if len(batch) >= BATCH_SIZE:
                        insert_batch(client, CLICKHOUSE_DATABASE, 'tic_tac_toe_states', batch, layer_name)
                        total_rows_inserted += len(batch)
                        batch = []
                
                pbar.update(1)
            
            # Insert remaining data
            if batch:
                insert_batch(client, CLICKHOUSE_DATABASE, 'tic_tac_toe_states', batch, layer_name)
                total_rows_inserted += len(batch)
    
    print(f"✓ Completed {layer_name}: {rows_processed} rows inserted")

print(f"\n{'='*60}")
print(f"Total rows inserted: {total_rows_inserted:,}")
print(f"{'='*60}")


In [ ]:
# Verify data insertion
result = client.execute(f"SELECT layer_name, COUNT(*) as count FROM {CLICKHOUSE_DATABASE}.tic_tac_toe_states GROUP BY layer_name ORDER BY layer_name")

print("\nData verification:")
print("-" * 40)
print(f"{'Layer Name':<20} {'Row Count':>15}")
print("-" * 40)
for layer_name, count in result:
    print(f"{layer_name:<20} {count:>15,}")
print("-" * 40)

total = client.execute(f"SELECT COUNT(*) FROM {CLICKHOUSE_DATABASE}.tic_tac_toe_states")[0][0]
print(f"{'Total':<20} {total:>15,}")


In [ ]:
# Sample query: Get some sample data
sample_data = client.execute(f"""
    SELECT * FROM {CLICKHOUSE_DATABASE}.tic_tac_toe_states 
    WHERE layer_name = 'layer_9' 
    LIMIT 5
""")

print("\nSample data from layer_9:")
for row in sample_data:
    print(row)


In [ ]:
# Close connection
client.disconnect()
print("\nDisconnected from ClickHouse.")
